# NLGM: non-linear Gaussian Model

## I. 背景
### I.1 LGM模型的限制 
- 用Linear Gaussian Model解决生成任务时，有一个核心假设：$x = \Lambda z + \epsilon$，也就是说，样本随机变量是与latent varible之间是线性关系。
  - 如下图所示：LGM可以表示上图的关系，但不能表示下面那张图。
  <img src="../pics/fa_1D.png" alt="alt text" width="500"/>
  <img src="../pics/LGM_limitation.png" alt="alt text" width="500"/>

### I.2 解决思路
1. 方法：
   1. 放松LGM的线性假设，取$x = f(z) + \epsilon$
      - <img src="../pics/NLGM_generator.png" alt="alt text" width="500"/>
   2. 同时用neural network作为$f(z)$，这样就能learn到非常复杂的结构

2. 如何生成与样本分布一样的数据
   - 作为generator的工作方式：从标准高斯分布中抽z，然后用f(z)将z投影到non-linear manifold上，最后加上高斯噪声。

## NLGM (也就是VAE)
### I. 模型
#### I.1 模型的假设条件
$$
\begin{align}
& x =f(z;\theta) + \epsilon=f_{\theta}(z) + \epsilon \, ，这里\theta是神经网络的weights\\
& z\sim N(0, I_p) \\
& \epsilon \sim N(0, \Psi_d )，简化取\Psi_d=\sigma^2I_d\\
& \epsilon与z相互独立
\end{align}
$$
  - 要learn的参数：神经网络中的参数$W, \sigma^2$

#### I.2 目标函数:MLE
$$ \begin{align}
max \, logL(\theta) 
& = max \Sigma_{i=1}^{n} logP(x_i) \\
& = max \Sigma_{i=1}^{n} log\int_z P(x_i, z)dz \\
& = max \Sigma_{i=1}^{n} log\int_z P(x_i| z)P(z)dz \\
\end{align}$$
- 分析：
  1. log sum形态的目标函数没办法直接解最优化问题。
  2. 这里是由latent variable的MLE估计，所以考虑用EM算法求解最优化问题。
#### I.3 一般EM算法的求解思路
1. 思路
   - 将最大化likelihood转化为最大化ELBO
   $$logP(x_i;\theta) = ELBO(x_i;Q,\theta)+D_{KL}(Q(z|x_i)||P(z|x_i))$$
   $$\begin{align}
& \because D_{KL}(Q(z|x_i)||P(z|x_i))  \ge 0 \\
& \therefore logP(x_i;\theta)
  \ge ELBO(x_i;Q,\theta)
\end{align}$$
   - 替代优化目标：
   $$max \Sigma_iELBO(x_i;Q,\theta)  = max \Sigma_iE_{z\sim Q(z)}log\frac{P(x_i, z;\theta)}{Q(z)}$$

2. 算法实现方式
   - E step：
     - 求latent variable的后验分布$P(z|x)$，补齐样本信息。
     - 操作方式：
       1. 对每个样本$x_i$，用$sample(P(z|x_i))$得到$对应的z_i$
       2. 将$(x_i,z_i)$代入ELBO。得到估计值：$$
       \begin{align}
\Sigma_iELBO(x_i;Q,\theta)  
& = \Sigma_iE_{z\sim Q(z)}log\frac{P(x_i, z;\theta)}{Q(z)} \\
& = \Sigma_ilog\frac{P(x_i, z_i;\theta)}{Q(z_i|x_i)}
\end{align}
       $$
   - M step：
     - 更新参数：$$\begin{align}
\theta_{t+1} & = \underset{\theta}{argmax}\Sigma_i ELBO(x_i;Q, \theta )\\
& = \underset{\theta}{argmax}\Sigma_ilog\frac{P(x_i, z_i;\theta)}{Q(z_i|x_i)} \\
& = \underset{\theta}{argmax}\Sigma_ilogP(x_i, z_i;\theta) \, ，\because 这里Q(z_i|x_i)固定，不随\theta变化\\
& = \underset{\theta}{argmax}\Sigma_ilogP(x_i| z_i;\theta) \, ，\because P(z_i)与\theta无关
\end{align}$$

3. NLGM问题：无法直接求解$P(z|x)$的显式解
   $$\begin{align}
P(z|x) & = \frac{P(x|z;\theta )P(z)}{P(x)} \\
& = \frac{P(x|z;\theta)P(z)}{\int_zP(x|z)P(z)dz} \\
& = \frac{N(f_\theta (z),\Psi)N(0,I)}{\int_z N(f_\theta (z),\Psi)N(0,I) dz}，没有显示解
\end{align} $$

#### I.4 VAE中使用EM算法的方式
1. 思路：仍然使用EM算法，只是在E step中由于无法直接计算$P(z|x)$，改用$Q(z|x)$来估计$P(z|x)$。取$Q(z|x)\sim N(\mu(x), \Sigma(x))$，用neural network来learn参数$\mu(x), \Sigma(x)$。

2. 迭代过程描述：下图中左边是正常EM的迭代过程，右图是VAE的迭代路径。可见：
   - 每次E step无法达到logP(x)的lower bound，但通过$Q(z|x)$逼近$P(z|x)$仍然会让ELBO的值有所增加.
   - 之后的M step还是可以在给定Q的条件下，通过参数迭代使ELBO达到它的最大值
   <img src="../pics/ELBO.png" width="500" height="360">   

### II. 调整后的EM算法

#### II.1 E step: 估计$Q(z|x)\sim N(\mu(x), \Sigma(x))$
1. 方法：通过$\underset{\varphi}{argmin}\, KL(Q_{\varphi}(z|x)||P(z|x;\theta))$来找逼近$P(z|x;\theta)$的$Q(z|x)$。
   - <font color=brown>参数说明
     1. $\varphi$表示用来learn $\mu(x), \Sigma(x)$的神经网络中的参数。
     2. $\theta$是当前步中$P(x|z)=N(f(z),\Psi)$中的参数的代称，实际包括神经网络$f(z)$中的权重和$\Psi$。
        - 因为：$$\begin{align}
P(z|x) & =\frac{P(x|z;\theta )P(z)}{\int_zP(x|z;\theta)P(z)dz} = \frac{N(f(z),\Psi)N(0,I)}{\int_zN(f(z),\Psi)N(0,I)dz} 
 \end{align}$$
       因此，$\theta$也是影响$P(z|x)$取值的参数。</font>

2. 求解$\underset{\varphi}{argmin}\, D_{KL}(Q_{\varphi}(z|x)||P_{\theta}(z|x))$
   - 根据$D_{KL}$的以下计算特征可知： *[证明见附录]*
$$\begin{align}
D_{KL}(Q_{{\color{red} \varphi}}(z|x)||P(z|x;{\color{blue} \theta}))
& = logP(x;{\color{blue} \theta} ) - ELBO(x; Q_{{\color{red} \varphi}}(z|x),{\color{blue} \theta}  ) \\
& = logP(x;{\color{blue} \theta}) - E_{z\sim Q_{{\color{red} \varphi}}(z|x)}log\frac{P(x|z;{\color{blue} \theta} )P(z)}{Q_{{\color{red}\varphi}}(z|x)} \\
\end{align}$$
     - **$\varphi$不影响$P(x)$，因此最小化KL divergence的目标等价于最大化ELBO。**
     - <font color=blue>**这与EM算法中M step的目标一样，只不过在M step中是通过调整参数$\theta$来实现max ELBO，这里是调整参数$\varphi$**</font>。<font color=green>**实际上这两个参数互相并不影响，都是各自直接影响ELBO的大小，因此可以合并求解这两个最优化问题。**</font>


3. 求解转化后的目标:$\underset{\varphi}{argmax}\, ELBO(x; Q_{\varphi}(z|x),\theta)$
   - ELBO分解：
     $$\begin{align}
& \underset{\color{red}{\varphi}}{argmax}\, ELBO(x; Q_{\color{red}{\varphi}}(z|x),\theta) \\
& = \underset{\color{red}{\varphi}}{argmax}\, E_{z\sim Q_{\color{red}{\varphi}}(z|x)}log\frac{P_{{\color{green} \theta} }(x|z)P(z)}{Q_{\color{red}{\varphi}}(z|x)} \\
& = \underset{\color{red}{\varphi}}{argmax}\, \int_zQ_{\color{red}{\varphi}}(z|x)log\frac{P_{\color{green}\theta}(x|z)P(z)}{Q_{\color{red}{\varphi}}(z|x)}dz\\
\end{align}$$
   - 从分解式中可以看出，$\varphi和\theta$影响的是ELBO中的不同组成部分。M step中也是以max ELBO为目标做参数$\theta$的估计。这里$\varphi和\theta$的取值并不对彼此产生影响，因此可以将两个优化问题合并完成。
   - <font color=green>**所以在E step中先不解最优化问题，将它与M step合并来解$\varphi$和$\theta$**</font>

4. 解优化问题的方法：将$ELBO(x_i;\varphi,\theta)$设为loss function，用SGD方法求解
   - **不同参数影响哪些分布的判断要注意**：
     - 从上面ELBO的分解式中来看，$\varphi$只影响$Q_{\varphi}(z|x)$,$\theta$只影响$P_{\theta}(x|z)$，且他们对$P(z)$都没有影响。
     - 如果式中每个分布都有显示表达式可以代入，那么没有问题。
     - 如果要用抽样方式估计$E_{z\sim Q_{\color{red}{\varphi}}(z|x)}log\frac{P_{{\color{green} \theta} }(x|z)P(z)}{Q_{\color{red}{\varphi}}(z|x)}$就要注意：<font color=red>**此时样本z是从$Q_{\varphi}(z|x)$中抽样得到的，因此式中所有z的取值都会受$\varphi$的影响，包括$P(z)和P(x|z)$。**</font>
   - [具体过程详见后文]

#### II.2 M step：用E step中最优化的结果进行抽样，补齐$(x_i, z_i)$，代入解max ELBO

1. 第一步：用$Q(z|x)$完成抽样$z_i=Sample(Q(z|x_i))$

2. 第二步：更新参数，最大化ELBO
$$\begin{align}
\theta_{t+1}
& = \underset{\theta}{argmax}\Sigma_i ELBO(x_i;Q, \theta )\\
& = \underset{\theta}{argmax}\Sigma_ilog\frac{P(x_i, z;\theta)}{Q(z|x_i)} \, ，这里Q(z|x)不受\theta影响\\
& = \underset{\theta}{argmax}\Sigma_ilogP(x_i|z_i;\theta)+logP(z_i) \, ，这里P(z)不受\theta影响\\
& = \underset{\theta}{argmax}\Sigma_ilogP(x_i|z_i;\theta) \, ，代入P(x|z)=N(f(z),\sigma^2I_d)\\
& = \underset{\theta}{argmax}\Sigma_i-\frac{d}{2}log\sigma^2-\frac{1}{2\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2  \\ 
& = \underset{\theta}{argmin}\Sigma_i dlog\sigma^2+\frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2 
\end{align}$$

#### II.3 合并优化目标
1. 合并优化目标
$$\begin{align}
& \underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax} \,\Sigma_iELBO(x; Q_{\color{red}{\varphi}}(z|x),{\color{Green} \theta} ) \\
& =\underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax} \,\Sigma_i\, E_{z\sim Q_{\color{red}{\varphi}}(z|x)}log\frac{P_{{\color{green} \theta} }(x|z)P(z)}{Q_{\color{red}{\varphi}}(z|x)}\\
& = \underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax} \,\Sigma_i\, \int_zQ_{\color{red}{\varphi}}(z|x)log\frac{P_{\color{green}\theta}(x|z)P(z)}{Q_{\color{red}{\varphi}}(z|x)}dz\\
& = \underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax}\,\Sigma_i \int_zQ_{\color{red}{\varphi}}(z|x)logP_{\color{green}\theta}(x|z) -\Sigma_i\,\int_zQ_{\color{red}{\varphi}}(z|x)log\frac{Q_{\color{red}{\varphi}}(z|x)}{P(z)}dz \\
& = \underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax}\,\Sigma_i E_{z\sim Q_{\color{red}{\varphi}}(z|x)}logP_{\color{green}\theta}(x|z) - \Sigma_iD_{KL}(Q_{\color{red}{\varphi}}(z|x)||P(z)) \\
\end{align}$$
   - 为方便构造Gradient Descent中的loss function，选择以下等价的目标表达式：
$$\begin{align}
\underset{\varphi,\theta}{argmin}\, \Sigma_i \int_zQ_{\color{red}{\varphi}}(z|x)log\frac{Q_{\color{red}{\varphi}}(z|x)}{P(z)}dz - \Sigma_i\,E_{z\sim Q_{\color{red}{\varphi}}(z|x)}logP_{\color{green}\theta}(x|z) \\
\end{align}$$

2. 第一项直接求解
   - 根据两个正态分布之间的KL divergence公式: *[证明见附录]*
     $$D_{KL}(p || q) = \frac{1}{2} \left[ \text{tr}(\Sigma_q^{-1} \Sigma_p) + (\mu_p - \mu_q)^T \Sigma_q^{-1} (\mu_p - \mu_q) - k + \log \left(\frac{|\Sigma_q|}{|\Sigma_p|}\right) \right]$$
     - 代入$p=Q_{\varphi}(z|x)，q=N(0,I)$:
       $$\begin{align}
D_{KL}(Q || N(0,I)) 
& = \frac{1}{2} \left[ \text{tr}(\Sigma_q^{-1} \Sigma_Q) + (\mu_Q - \mu_q)^T \Sigma_q^{-1} (\mu_Q - \mu_q) - d + \log \left(\frac{|\Sigma_q|}{|\Sigma_Q|}\right) \right] \\
& = \frac{1}{2} [ \text{tr}(\Sigma_{\varphi}(x)) +\mu_{\varphi}(x)^T\mu_{\varphi}(x)- d - \log |\Sigma_{\varphi}(x)|  ]
\end{align}$$

3. $P(x|z)$是neural network，所以第二项无法得到显式表达式，只能用抽样方法估计
   - 样本量很大时，每个$x_i$可以只sample1次，取1个$z_i$:
     $$\Sigma_iE_{z\sim Q_{\varphi}(z|x)}logP(x_i|z;\theta) = \Sigma_i logP(x_i|z_i;\theta)$$
   - 展开概率表达式有：
     $$\begin{align}
logP(x_i|z_i;\theta) = 
-\frac{p}{2}log\sigma^2-\frac{1}{2\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2
\end{align}$$

4. 代入可得优化目标:
$$\begin{align}
\underset{\varphi,\theta}{argmax} \,\Sigma_iELBO 
&= \underset{\varphi,\theta}{argmin}\, \Sigma_i\int_zQ_{\color{red}{\varphi}}(z|x_i)log\frac{Q_{\color{red}{\varphi}}(z|x_i)}{P(z)}dz - \Sigma_i\,E_{z\sim Q_{\color{red}{\varphi}}(z|x_i)}logP_{\color{green}\theta}(x_i|z) \\
&= \underset{\varphi,\theta}{argmin}\, \Sigma_i\frac{1}{2} [ \text{tr}(\Sigma_{\varphi}(x_i)) +\mu_{\varphi}(x_i)^T\mu_{\varphi}(x_i)- d - \log |\Sigma_{\varphi}(x_i)|  ] \\
& -(-\frac{d}{2}log\sigma^2-\frac{1}{2\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2) \\
&= \underset{\theta,\varphi}{argmin}\,\Sigma_i \text{tr}(\Sigma_{\varphi}(x_i)) +\left \| \mu_{\varphi}(x_i) \right \|^2 - \log|\Sigma_{\varphi}(x_i)| + dlog\sigma^2 + \frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2
\end{align}$$

#### II.4 更新后的迭代算法
- 将E step和M step中的优化问题合并，重新排序后的算法
  - 初始化参数
  - **迭代：**
    - <font color=blue>
第t步：用当前$Q(z|x,\varphi)$抽样，补齐所有样本信息为$(x_i, z_i)$</font>
    - <font color=blue>
第t+1步：用SGD方法同步更新所有参数。$$\theta_{t+1}, \varphi_{t+1}
= \underset{\theta,\varphi}{argmax}\Sigma_i ELBO(x_i;Q_{\varphi }(z|x), \theta )$$</font>

### III. VAE的优化求解过程
#### III.1 第一步，抽样$Q_{\varphi}(z|x)=N(\mu(x), \Sigma(x);\varphi)$
- 直接抽样存在的问题
  1. 抽样结果方差大
  2. 后面用SGD求解优化问题的时候很难对“抽样”操作求梯度
- <font color=green>**solution：用重参数技巧做抽样**</font>
  - 方法：将$Q_{\varphi}(z|x)=N(\mu(x), \Sigma(x);\varphi)$转化成等价的另一个分布：$$\begin{align}
z & = \mu_{\varphi}(x) + \left(\Sigma_{\varphi}(x)\right)^{\frac{1}{2}}\varepsilon  \\
\varepsilon  & \sim N(0,I_p) \, ,这里p=dim(z)
\end{align}$$
  - 效果：通过将随机性转移到$\varepsilon$上，使得对原参数$\varphi$的梯度求解过程中不再有随机性。并且也降低了抽样的方差。**[为什么能降低抽样方差没有说明？？？]**

#### III.2  第二步，更新参数$\varphi$和$\theta$

1. **优化目标**
   - 原目标
$$\begin{align}
\theta_{t+1} & = \underset{\theta}{argmin}\,\Sigma_i dlog\sigma^2+\frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2 \\
\varphi_{t+1} & = \underset{\varphi}{argmin}\,\Sigma_i \text{tr}(\Sigma_{\varphi}(x_i)) +\left \| \mu_{\varphi}(x_i) \right \|^2  - \log|\Sigma_{\varphi}(x_i)| + \frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2
\end{align}$$
   - 合并后的目标 
   $$\underset{\theta,\varphi}{argmin}\,\Sigma_i \text{tr}(\Sigma_{\varphi}(x_i)) +\left \| \mu_{\varphi}(x_i) \right \|^2 - \log|\Sigma_{\varphi}(x_i)| + dlog\sigma^2 + \frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2$$
     - <font color=green>注意：$f_{\theta}(z_i)$中的$z_i$是$Q_{\varphi}(z|x)$抽样的结果，所以求梯度时要考虑这个影响。</font>

2. **用优化目标构造Loss function**
$$L=\Sigma_i \text{tr}(\Sigma_{\varphi}(x_i)) +\left \| \mu_{\varphi}(x_i) \right \|^2 - \log|\Sigma_{\varphi}(x_i)| + dlog\sigma^2 + \frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2$$

3. **求解用到的梯度公式**
$$\begin{align}
\nabla_{\varphi} L &= \Sigma_i \nabla_{\varphi}\text{tr}(\Sigma_{\varphi}(x_i)) +\left \| \mu_{\varphi}(x_i) \right \|^2 - \log|\Sigma_{\varphi}(x_i)|+ \nabla_{z}\frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2\nabla_{\varphi}z \\
\nabla_{\theta} L &= \Sigma_i \nabla_{\theta}\left(dlog\sigma^2 + \frac{1}{\sigma^2}\left \| x_i-f_{\theta}(z_i) \right \|^2 \right)\\
\end{align}$$

### IV. Inference阶段
- 生成(图片)的方式：
   - step 1：用$z\sim N(0,I)$抽一个z
   - step 2：用$f_{\theta}(z)$得到结果，通常inference中，这步不再加噪音


### V. VAE模型小结
#### V.1 模型特征
1. 模型是encoder-decoder结构，Encoder是$z|x\sim N(\mu_{\varphi}(x),\Sigma_{\varphi}(x))$的部分，Decoder是$x|z\sim N(f_{\theta}(z), \sigma^2I_p)$的部分。
   - 完成训练后，扔掉encoder的部分，保留decoder就是generator，用于完成inference任务。
2. VAE本质上就是LGM的非线性扩展。由于引入neural network后不能获得后验分布的显式表达式，因此在LGM的基础上做了几点调整：
   1. 引入了另一个多元高斯分布来逼近后验分布。
   2. 为了让整个过程可以顺利求梯度，因此在抽样$(x_i,z_i)$的时候用了重参数技巧
3. 同样因为后验分布没有显式表达式，P(x)也无法得到显式表达式，因此VAE能通过抽样来generate P(x)的样本，但是却不能在给定x的条件下，判断它的概率大小，因此不能做异常检测。

#### V.2 优化目标的含义
$$\begin{align}
& \underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax} \,\Sigma_iELBO(x_i; Q_{\color{red}{\varphi}}(z|x),{\color{Green} \theta} ) \\
& = \underset{{\color{red}{\varphi}},{\color{Green} \theta} }{argmax}\,\Sigma_i \underset{En/Decoder\, target}{\underbrace{E_{z\sim Q_{\color{red}{\varphi}}(z|x_i)}logP_{\color{green}\theta}(x_i|z)} }  - \Sigma_i \underset{regularization}{\underbrace{D_{KL}(Q_{\color{red}{\varphi}}(z|x_i)||P(z))}}   \\
\end{align}$$

1. 目标中第一部分表达的是：
     - 先用$Q(z|x)$得到样本$x_i$的latent variable$z_i$的取值，完成x信息的encoder。
     - 再用$P(f(z))$计算$x_i$的生成概率，完成对z中信息的Decoder。
     - $max E_{z\sim Q_{\color{red}{\varphi}}(z|x_i)}logP_{\color{green}\theta}(x_i|z)$整个可以看做最大化$x_i$的reconstruction probability
2. 第二部分让后验分布逼近先验分布，有两个作用：
     - 一是regulator，防止encoder过拟合
     - 二是让decoder作为generator工作的时候，从先验假设$P(z)$中得到的抽样尽可能逼近training set中样本群体的latent variable取值$Q_{\varphi}(z|x_i)$。这样才能保证生成的图片与training set中的图片尽可能接近，这也是模型泛化能力的体现。

## 附录
### 附1. 计算两个正态分布之间的KL divergence公式
- 给定两个k维多元正态分布p和q：
  - $ p(x) = \mathcal{N}(\mu_p, \Sigma_p) $
  - $ q(x) = \mathcal{N}(\mu_q, \Sigma_q) $
  - $D_{KL}(p || q) = \int p(x) \log \frac{p(x)}{q(x)} \, dx$
- 他们之间的KL divergence可以用下面公式计算：
$$D_{KL}(p || q) = \frac{1}{2} \left[ \text{tr}(\Sigma_q^{-1} \Sigma_p) + (\mu_p - \mu_q)^T \Sigma_q^{-1} (\mu_p - \mu_q) - k + \log \left(\frac{|\Sigma_q|}{|\Sigma_p|}\right) \right]$$
- 证明：

#### 已知：
$$
\begin{align}
p(x) = \frac{1}{(2\pi)^{k/2} |\Sigma_p|^{1/2}} \exp\left(-\frac{1}{2}(x - \mu_p)^T \Sigma_p^{-1} (x - \mu_p)\right) \\
q(x) = \frac{1}{(2\pi)^{k/2} |\Sigma_q|^{1/2}} \exp\left(-\frac{1}{2}(x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q)\right)
\end{align}
$$

#### 代入：
$$D_{KL}(p || q) = \int p(x) \left( \log p(x) - \log q(x) \right) \, dx$$

**计算logP(x)和logq(x)** ：
$$\log p(x) = -\frac{k}{2} \log(2\pi) - \frac{1}{2} \log |\Sigma_p| - \frac{1}{2}(x - \mu_p)^T \Sigma_p^{-1} (x - \mu_p)$$
$$\log q(x) = -\frac{k}{2} \log(2\pi) - \frac{1}{2} \log |\Sigma_q| - \frac{1}{2}(x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q)$$

**代入KL divergence公式** ：
$$\begin{align}
D_{KL}(p || q) 
& = \frac{1}{2}\int p(x) \left(-\log |\Sigma_p| -(x - \mu_p)^T \Sigma_p^{-1} (x - \mu_p)  + \log |\Sigma_q| + (x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q) \right) dx \\
& = -\frac{1}{2} \int p(x) \log |\Sigma_p| \, dx + \frac{1}{2} \int p(x) \log |\Sigma_q| \, dx \\
& - \frac{1}{2} \int p(x) (x - \mu_p)^T \Sigma_p^{-1} (x - \mu_p) \, dx \\
& + \frac{1}{2} \int p(x) (x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q) \, dx
\end{align}$$

#### 分别计算上面的三个部分
1. **第一部分**: 
Since:
$$\int p(x) \, dx = 1$$
Thus,
$$-\frac{1}{2} \int p(x) \log |\Sigma_p| \, dx = -\frac{1}{2} \log |\Sigma_p|$$
$$\frac{1}{2} \int p(x) \log |\Sigma_q| \, dx = \frac{1}{2} \log |\Sigma_q|$$

2. **先求第三部分，第二部分是它的特殊情况**: 此时求期望所使用的x的分布是p(x):
   - 结果：$$E[(x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q)]= tr(\Sigma_q^{-1}\Sigma_p) - (\mu_p - \mu_q)^T \Sigma_q^{-1} (\mu_p - \mu_q)$$

   - 分析：
     - 第一步：展开目标求解式$$E[(x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q)] = E[x^T \Sigma_q^{-1} x] - 2\mu_q^T \Sigma_q^{-1} E[x] + \mu_q^T \Sigma_q^{-1} \mu_q$$

     - 第二步：求$E[x^T \Sigma_q^{-1} x]$
       - 正态分布有两个性质：
       1. $Exx^T= \Sigma + \mu \mu^T$ [详见证明1]
       2. $
E[x^T \Sigma_q^{-1} x] = \text{tr}(\Sigma_q^{-1} E[xx^T])$ [详见证明2]
       - 将B代入A可得：$$\begin{align}
E[x^T \Sigma_q^{-1} x] & = tr(\Sigma_q^{-1}(\Sigma_p + \mu_p \mu_p^T)) \\
& = tr(\Sigma_q^{-1}\Sigma_p + \Sigma_q^{-1}\mu_p \mu_p^T) \\
& = tr(\Sigma_q^{-1}\Sigma_p) + tr(\Sigma_q^{-1}\mu_p \mu_p^T) \\
& = tr(\Sigma_q^{-1}\Sigma_p) + tr(\mu_p^T\Sigma_q^{-1}\mu_p ) \\
& \because \Sigma_q^{-1} is \, a \, scalar \\
& = tr(\Sigma_q^{-1}\Sigma_p) + \mu_p^T\Sigma_q^{-1}\mu_p \\
\end{align}$$
     
     - 将结果代入目标式：$$\begin{align}
E[(x - \mu_q)^T \Sigma_q^{-1} (x - \mu_q)] 
& = E[x^T \Sigma_q^{-1} x] - 2\mu_q^T \Sigma_q^{-1} E[x] + \mu_q^T \Sigma_q^{-1} \mu_q \\
& = tr(\Sigma_q^{-1}\Sigma_p) + \mu_p^T\Sigma_q^{-1}\mu_p
- 2\mu_q^T \Sigma_q^{-1}\mu_p + \mu_q^T \Sigma_q^{-1} \mu_q \\
& = tr(\Sigma_q^{-1}\Sigma_p) - (\mu_p - \mu_q)^T \Sigma_q^{-1} (\mu_p - \mu_q)
\end{align}$$
        

3. **第二部分**: 
   - 直接代入第三部分计算结果：$$E[(x - \mu_p)^T \Sigma_p^{-1} (x - \mu_p)] = \text{tr}(\Sigma_p^{-1} \Sigma_p) + (\mu_p - \mu_p)^T \Sigma_p^{-1} (\mu_p - \mu_p) = k$$     

#### 证明1：$Exx^T= \Sigma + \mu \mu^T$
$$\begin{align}
x\sim N(\mu, \Sigma)时有：
\Sigma & = E(x-\mu)(x-\mu)^T\\
& = E(xx^T-2ux^T+uu^T) \\
& = Ex^Tx-u^Tu \\
\therefore Exx^T&= \Sigma + \mu \mu^T 
\end{align}$$

#### 证明2：$
E[x^T \Sigma_q^{-1} x] = \text{tr}(\Sigma_q^{-1} E[xx^T])$
$$\begin{align}
& \because x^T \Sigma_q^{-1} x是scalar \\
& \therefore x^T \Sigma_q^{-1} x = tr(x^T\Sigma_q^{-1} xx^T )=tr(\Sigma_q^{-1} xx^T) \\
& \therefore E[x^T \Sigma_q^{-1} x] = E[tr(\Sigma_q^{-1} xx^T)]=tr(E[\Sigma_q^{-1} xx^T])
=tr(\Sigma_q^{-1}E[xx^T])
\end{align}$$

### 附2：证明 
$$logP(x;\theta) = ELBO(x;Q(z|x),\theta)+D_{KL}(Q(z|x)||P(z|x;\theta))$$
- 证明：
$$\begin{align}
D_{KL}(Q(z|x_i)||P(z|x_i;{\color{Red} \theta}))
& = \int_zQ(z|x_i)log\frac{Q(z|x_i)}{P(z|x_i;{\color{Red} \theta})}dz \\
& = \int_zQ(z|x_i)log\frac{Q(z|x_i)P(x_i;{\color{Red} \theta})}{P(z,x_i;{\color{Red} \theta})}dz  \\
& = \int_zQ(z|x_i)\left[ logP(x_i;{\color{Red} \theta}) -log\frac{P(z,x_i;{\color{Red} \theta})}{Q(z|x_i)}\right]dz  \\
& = logP(x_i;{\color{Red} \theta})\int_zQ(z|x_i)dz - \int_zQ(z|x_i)log\frac{P(z,x_i;{\color{Red} \theta})}{Q(z|x_i)}dz \\
& = logP(x_i;{\color{Red} \theta}) - E_{z\sim Q(z|x_i)}log\frac{P(z,x_i;{\color{Red} \theta})}{Q(z|x_i)} \\
& = logP(x_i;{\color{Red} \theta}) - E_{z\sim Q(z|x_i)}log\frac{P(x_i|z;{\color{Red} \theta} )P(z)}{Q(z|x_i)} \\
& = logP(x_i;{\color{Red} \theta} ) - ELBO(x_i; Q,{\color{Red} \theta}  )
\end{align}$$